# Try to use unsloth to load framework

In [1]:
import json
import io
import os


def _make_r_io_base(f, mode: str):
    if not isinstance(f, io.IOBase):
        f = open(f, mode=mode)
    return f


def jload(f, mode="r"):
    """Load a .json file into a dictionary."""
    f = _make_r_io_base(f, mode)
    jdict = json.load(f)
    f.close()
    return jdict


def test_instruction_data_loader(
    test_instruction_path="./test_instructions.json",
):
    """
    load test instruction (about 100 samples)
    """
    test_instruction_data = []
    if os.path.exists(test_instruction_path):
        test_instruction_data = jload(test_instruction_path)
        print(f"Loaded {len(test_instruction_data)} test instructions")
        return test_instruction_data
    else:
        return []

In [4]:
data = test_instruction_data_loader()

Loaded 107 test instructions


In [14]:
data[2]

{'instruction': 'Construct a URScript function that moves the robot TCP to a new pose relative to the current TCP pose, then defines a new TCP offset, effectively transforming the new pose given relative to the base frame to be relative to the new tcp offset. Ensure no motion occurs in this definition and that the new definition is made according to the input variables.',
 'input': '```URscript\nrelative_position = p[0.1, 0.1, 0.1, 0, 0, 0]\nnew_tcp_offset_p = p[0.0, 0.0, 0.1, 0, 0, 0]\n```',
 'output': '```URscript\ndef update_tcp(relative_position, new_tcp_offset_p):\n  local current_pose = get_actual_tcp_pose()\n  local target_pose = pose_add(current_pose, relative_position)\n  movel(target_pose, a=0.2, v=0.5)\n  set_tcp(new_tcp_offset_p)\nend\n```',
 'most_similar_instructions': {'Define a function in URscript that repeatedly moves the robot to a position relative to its current TCP, based on a predefined offset. The movement is continuous, using specified acceleration and velocity

In [9]:
for key in data[0]:
    print(key)    

instruction
input
output
most_similar_instructions
avg_similarity_score


In [10]:
# try to merge instruction and output
new_samples = data[0]

In [20]:
def merge_instruction_input(data_dict):
    """
    Merges the instruction and input fields of a dictionary using a specific format.

    Args:
      data_dict: A dictionary containing 'instruction' and 'input' keys.

    Returns:
      A string representing the merged instruction and input, or None if the required keys are missing.
    """
    if 'instruction' not in data_dict or 'input' not in data_dict:
        return None
    instruction = data_dict['instruction']
    input_str = data_dict['input']
    merged_prompt = f"{instruction}[[\nYour input is:\n{input_str}]]"
    return merged_prompt

In [21]:
a = merge_instruction_input(data[2])
a

'Construct a URScript function that moves the robot TCP to a new pose relative to the current TCP pose, then defines a new TCP offset, effectively transforming the new pose given relative to the base frame to be relative to the new tcp offset. Ensure no motion occurs in this definition and that the new definition is made according to the input variables.[[\nYour input is:\n```URscript\nrelative_position = p[0.1, 0.1, 0.1, 0, 0, 0]\nnew_tcp_offset_p = p[0.0, 0.0, 0.1, 0, 0, 0]\n```]]'

In [22]:
print(a)

Construct a URScript function that moves the robot TCP to a new pose relative to the current TCP pose, then defines a new TCP offset, effectively transforming the new pose given relative to the base frame to be relative to the new tcp offset. Ensure no motion occurs in this definition and that the new definition is made according to the input variables.[[
Your input is:
```URscript
relative_position = p[0.1, 0.1, 0.1, 0, 0, 0]
new_tcp_offset_p = p[0.0, 0.0, 0.1, 0, 0, 0]
```]]


In [23]:
print(data[2]['output'])

```URscript
def update_tcp(relative_position, new_tcp_offset_p):
  local current_pose = get_actual_tcp_pose()
  local target_pose = pose_add(current_pose, relative_position)
  movel(target_pose, a=0.2, v=0.5)
  set_tcp(new_tcp_offset_p)
end
```
